In [57]:
import pandas as pd
from datetime import datetime
import re

import os
from docx import Document
from docx2pdf import convert

import warnings
warnings.filterwarnings('ignore')

# remove the doxc
# os.remove("rma_inv_template.docx")

In [2]:

inv_info = pd.read_excel("inv_info_sample.xlsx")
#rma_data['Row_Number'] = rma_data.groupby('Date').cumcount() + 1
#rma_data[rma_data['Inv_No'].isna()]

In [8]:
inv_no_list = inv_info['Inv_No'].unique()

In [13]:
inv_no_list

array(['SAMCO_1', 'SAMCO_2', 'SAMCO_3', 'SAMCO_4'], dtype=object)

In [11]:
item_dict = {}
for i in range(1, 16):
    item_dict['ITEM' + str(i)] = ""
    item_dict['DETAIL' + str(i)] = ""
    item_dict['UNITPRICE' + str(i)] = ""
    item_dict['QUANTITY' + str(i)] = ""
    item_dict['AMOUNT' + str(i)] = ""

In [123]:
inv_no = 'SAMCO_1'

In [124]:
use_info = inv_info[inv_info['Inv_No'] == inv_no].reset_index(drop = True)

item_dict = {}
for i in range(1, 16):
    try:
        item_dict['ITEM' + str(i)] = use_info.loc[i-1, 'Item']
        item_dict['DETAIL' + str(i)] = use_info.loc[i-1, 'Detail']
        item_dict['UNITPRICE' + str(i)] = use_info.loc[i-1, 'Unit Price']
        item_dict['QUANTITY' + str(i)] = use_info.loc[i-1, 'Quantity']
        item_dict['AMOUNT' + str(i)] = use_info.loc[i-1, 'Unit Price'] * use_info.loc[0, 'Quantity']
    except:
        item_dict['ITEM' + str(i)] = ""
        item_dict['DETAIL' + str(i)] = ""
        item_dict['UNITPRICE' + str(i)] = ""
        item_dict['QUANTITY' + str(i)] = ""
        item_dict['AMOUNT' + str(i)] = ""

In [137]:
import re
from docx import Document

def replace_text_in_doc_table(item_dict, inv_no):
    doc = Document('inv_template.docx')

    for table in doc.tables:
        for row in table.rows:
            for cell in row.cells:
                for para in cell.paragraphs:
                    # 合并所有 run 的文本
                    full_text = ''.join([run.text for run in para.runs])

                    # 替换文本
                    for i in range(1, 16):
                        for name in ['ITEM', 'DETAIL', 'UNITPRICE', 'QUANTITY', 'AMOUNT']:
                            full_text = re.sub(r'\b' + re.escape(name + str(i)) + r'\b',
                                               str(item_dict.get(name + str(i), '')), full_text)

                    para.clear()
                    para.add_run(full_text)

    new_doc_path = inv_no + '.docx'
    doc.save(new_doc_path)

# 调用函数
replace_text_in_doc_table(item_dict, inv_no)

In [8]:
for i in rma_data[rma_data['Inv_No'].isna()].index:
    [customer, amount, doc_date, row_num, description] = rma_data.loc[i, ['Customer', 'Amount', 'Date', 'Row_Number', 'Description']]
    amount = '{0:.2f}'.format(amount)
    inv_no = "LURMA" + doc_date.strftime('%Y%m%d') + str(row_num).zfill(3)
    
    # create the pdf invoice
    replace_text_in_table(customer, amount, doc_date, inv_no)
    convert(inv_no + ".docx")
    os.remove(inv_no + ".docx")
    
    rma_data.loc[i, 'inv_no'] = inv_no
    print(inv_no + " generated successfully!")